In [ ]:
pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 481.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=ba7e2b52651b91c386d6f0d2f5ca797679598b601aa6adcd0566ca8c4aba2a1d
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
pip install nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import re
import string
import nltk
import gensim
import fasttext
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from nltk.corpus import stopwords
from nltk import bigrams
from tensorflow import keras
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from tensorflow.keras.models import Sequential
from gensim.models import Word2Vec, KeyedVectors
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [ ]:
pip install PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.2 MB/s eta 0:00:00


## **Ekstrak teks PDF**

Ekstraksi dilakukan agar segala tulisan didalam file dapat diambil untuk selanjutnya diproses. Ekstraksi dilakukan dengan mencari per kalimat yang selanjutnya ditampung ke dalam kolom 'Kalimat' dalam dataframe (df)

In [ ]:
import PyPDF2
import pandas as pd
def extract_sentences (pdf_path):
    data = {'Kalimat': []}

    with open(pdf_path, 'rb') as file:
      pdf_reader = PyPDF2.PdfReader (file)

      for page_num in range(len (pdf_reader.pages)):
        page=pdf_reader.pages [page_num]
        text= page.extract_text()

        # Split text into sentences
        sentences = text.split('.')
        # Add sentences to the data dictionary
        data[ 'Kalimat'].extend(sentences)
    return pd.DataFrame (data)
# Example usage
pdf_path='/content/MALIN_KUNDANG.pdf'
df = extract_sentences(pdf_path)
# Display the DataFrame print (df)
df


,Kalimat
0,"MALIN KUNDANG \nPada suatu waktu, hiduplah seb..."
1,"Keluarga tersebut terdiri \ndari ayah, ibu da..."
2,Karena kondisi keuangan keluarga yang \nmempr...
3,\nMaka tinggallah si Malin dan ibunya di gubug...
4,"Semingg u, dua minggu, sebulan, dua \nbulan b..."
...,...
3734,Tubuh Prabu Dewata Cengkar dilempar Aji Saka ...
3735,\nAji Saka kemudian dinobatkan menjadi raja M...
3736,I a memboyong ayahnya ke \nistana
3737,"Berkat pemerintahan yang adil dan bijaksana, ..."


## **Pre-processing**

Pre-processing dilakukan untuk mempermudah pengolahan teks selanjutnya. Dilakukan dengan beberapa tahap seperti, menghapus tanda baca, angka, konversi huruf kecil, tokenisasi, menghapus stopwords. Sehingga, nantinya teks lebih efektif dan efisien untuk dianalisis.

In [ ]:
nltk.download('stopwords')
def clean_text(input_text):
    # Menghapus tanda baca
    translator = str.maketrans("", "", string.punctuation)
    text_without_punct = input_text.translate(translator)

    # Menghapus angka
    text_without_numbers = re.sub(r'\d', '', text_without_punct)

    # Mengonversi huruf kecil
    cleaned_text = text_without_numbers.lower()

    # Tokenisasi teks
    tokens = nltk.word_tokenize(cleaned_text)

    # Menghapus stopwords
    stop_words = set(stopwords.words('indonesian'))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    # Menggabungkan kembali teks dari token yang telah difilter
    cleaned_text = ' '.join(filtered_tokens)

    return cleaned_text


df['Kalimat'] = df['Kalimat'].apply(clean_text)
print(df)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


                                                Kalimat
0     malin kundang hiduplah keluarga nelayan pesisi...
1        keluarga ayah anak lakilaki nama malin kundang
2     kondisi keuangan keluarga memprihatinkan sang ...
3                      tinggallah si malin ibunya gubug
4     semingg u minggu sebulan ayah malin kampung ha...
...                                                 ...
3734  tubuh prabu dewata cengkar dilempar aji saka j...
3735            aji saka dinobatkan raja medang kamulan
3736                       i a memboyong ayahnya istana
3737  berkat pemerintahan adil bijaksana aji saka me...
3738                                                   

[3739 rows x 1 columns]


## **One Hot Encoding**

Mengubah data kategorikal menjadi sebuah vektor biner dengan nilai 1 pada kategori yang sesuai dan 0 untuk kategori lainnya.

In [ ]:
ONE_HOT=pd.get_dummies(df['Kalimat'].str.split(expand=True).stack(), drop_first=True).groupby(level=0).max()
ONE_HOT['Kalimat']=df['Kalimat']
ONE_HOT

,aaa,aaaa,aan,aat,abad,abadi,abangnya,abdi,abdinya,abu,...,yelamatkan,yet,yik,yikan,yir,yosaku,yuk,yut,zam,Kalimat
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,malin kundang hiduplah keluarga nelayan pesisi...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,keluarga ayah anak lakilaki nama malin kundang
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,kondisi keuangan keluarga memprihatinkan sang ...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,tinggallah si malin ibunya gubug
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,semingg u minggu sebulan ayah malin kampung ha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3733,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,prabu dewata cengkar marah serban aji s aka me...
3734,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,tubuh prabu dewata cengkar dilempar aji saka j...
3735,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,aji saka dinobatkan raja medang kamulan
3736,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,i a memboyong ayahnya istana


## **Hash Vectoring**

Mengubah data atau teks non-numerik menjadi representasi numerik (biasanya dalam bentuk bilangan bulat atau vektor biner) dengan cara yang terdistribusi secara acak. <br>
Fungsi hash memetakan data masukan (misalnya, kata atau objek) ke nilai hash yang sesuai, yang kemudian dapat digunakan sebagai representasi numerik dari data tersebut.

In [ ]:
import hashlib
def hash_vectoring(text):
    # Inisialisasi vektor dengan nilai 0
    vector = [0] * vector_size

    # Konversi teks menjadi hash
    hashed_text = hashlib.sha256(text.encode()).hexdigest()

    # Ambil sebagian dari hash (sesuai dengan panjang vektor)
    hash_subset = hashed_text[:vector_size]

    # Konversi hash menjadi bilangan bulat (integer)
    hash_integer = int(hash_subset, 16)

    # Modulus hash dengan ukuran vektor untuk mendapatkan indeks
    index = hash_integer % vector_size

    # Set nilai indeks vektor menjadi 1
    vector[index] = 1

    return vector

vector_size = 15
df_HASH = df["Kalimat"].apply(hash_vectoring)
print(df_HASH)

0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
1       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
                            ...                      
3734    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3735    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3736    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3737    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3738    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Name: Kalimat, Length: 3739, dtype: object


## **Co-Occurence Matriks**

In [ ]:
# Membuat dataset menjadi list dalam lists (per kalimat dijadikan list)
df_lists = df['Kalimat'].apply(lambda x: x.split()).tolist()
df_lists[1:5]

[['keluarga', 'ayah', 'anak', 'lakilaki', 'nama', 'malin', 'kundang'],
 ['kondisi',
  'keuangan',
  'keluarga',
  'memprihatinkan',
  'sang',
  'ayah',
  'memutuskan',
  'mencari',
  'nafkah',
  'negeri',
  'seberang',
  'mengarungi',
  'lautan',
  'luas'],
 ['tinggallah', 'si', 'malin', 'ibunya', 'gubug'],
 ['semingg',
  'u',
  'minggu',
  'sebulan',
  'ayah',
  'malin',
  'kampung',
  'halamannya']]

In [ ]:
import numpy as np
import nltk
from nltk import bigrams
import itertools
import pandas as pd

def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    vocab_to_index = {word: i for i, word in enumerate(vocab)}

    # Create bigrams from all words in corpus
    bi_grams = list(bigrams(corpus))

    # Frequency distribution of bigrams ((word1, word2), num_occurrences)
    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    # Initialise co-occurrence matrix
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab)))

    # Loop through the bigrams taking the current and previous word,
    # and the number of occurrences of the bigram.
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]
        pos_current = vocab_to_index[current]
        pos_previous = vocab_to_index[previous]
        co_occurrence_matrix[pos_current][pos_previous] = count

    co_occurrence_matrix = np.matrix(co_occurrence_matrix)

    # Return the matrix and the index
    return co_occurrence_matrix, vocab_to_index

merged = list(itertools.chain.from_iterable(df_lists))
matrix, vocab_to_index = co_occurrence_matrix(merged)

CoMatrixFinal = pd.DataFrame(matrix, index=vocab_to_index, columns=vocab_to_index)
print(CoMatrixFinal)

             mengadakan  tingkat  anakanak  pemuda  kesal  langgang  perompak  \
mengadakan          0.0      0.0       0.0     0.0    0.0       0.0       0.0   
tingkat             0.0      0.0       0.0     0.0    0.0       0.0       0.0   
anakanak            0.0      0.0       0.0     0.0    0.0       0.0       0.0   
pemuda              0.0      0.0       0.0     0.0    0.0       0.0       0.0   
kesal               0.0      0.0       0.0     0.0    0.0       0.0       0.0   
...                 ...      ...       ...     ...    ...       ...       ...   
lambat              0.0      0.0       0.0     0.0    0.0       0.0       0.0   
ungsu               0.0      0.0       0.0     0.0    0.0       0.0       0.0   
tom                 0.0      0.0       0.0     0.0    0.0       0.0       0.0   
berkelakuan         0.0      0.0       0.0     0.0    0.0       0.0       0.0   
rampokannya         0.0      0.0       0.0     0.0    0.0       0.0       0.0   

             prab  bag  pun

Baris dan kolom dalam matriks ini mewakili kata-kata dalam korpus yang digunakan.<br>
Setiap sel dalam matriks berisi angka yang menunjukkan seberapa sering dua kata muncul bersama-sama dalam konteks yang telah ditentukan.<br>
Nilai diagonal utama (misalnya, "lambat" di baris dan kolom sekian) umumnya diatur ke nol, karena ini menunjukkan seberapa sering sebuah kata muncul bersama dengan dirinya sendiri dalam konteks yang sama, yang biasanya tidak relevan.<br>
Nilai sel yang lebih besar menunjukkan bahwa kata-kata tersebut cenderung muncul bersama lebih sering dalam konteks yang sama. Contohnya, di baris "lapar" dan kolom "haus", nilai adalah 5 (jika dilihat dari file excelnya), menunjukkan bahwa kata "lapar" dan "haus" muncul bersama dalam lima konteks.<br>

## **WORD2VEC**

In [ ]:
# Membangun model Word2Vec
corpus=df['Kalimat']
tokenized_corpus = [sentence.split() for sentence in corpus]
model_w2v = Word2Vec(tokenized_corpus, vector_size=150, window=25, min_count=1, sg=1)

In [ ]:
words=list(model_w2v.wv.index_to_key)
vector_W2V = [model_w2v.wv[word] for word in words]
vector_W2V = np.array(vector_W2V)

In [ ]:
words[1:10]

['raja', 'si', 'sang', 'kancil', 'anak', 'puteri', 'istana', 'a', 'orang']

In [ ]:
vec_word2vec=pd.DataFrame(vector_W2V)
print(vec_word2vec)

           0         1         2         3         4         5         6    \
0    -0.114086 -0.079794 -0.030212 -0.066008  0.164640 -0.218131 -0.046164   
1    -0.099754 -0.075718 -0.038167 -0.065102  0.143348 -0.191161 -0.047286   
2    -0.094924 -0.042961 -0.038607 -0.089244  0.163655 -0.226439 -0.056719   
3    -0.094524 -0.059426 -0.033376 -0.088737  0.153762 -0.220889 -0.070693   
4    -0.069247 -0.056096 -0.046556 -0.083823  0.152241 -0.203306 -0.060706   
...        ...       ...       ...       ...       ...       ...       ...   
4879 -0.014075 -0.008424 -0.014014 -0.018467  0.033933 -0.029483 -0.006715   
4880 -0.005460 -0.006171 -0.008373 -0.008417  0.010040 -0.021307 -0.009938   
4881 -0.009584 -0.004450 -0.004733 -0.007926  0.010122 -0.024938 -0.007555   
4882 -0.005578 -0.007537 -0.004540 -0.012707  0.009462 -0.015897 -0.010643   
4883 -0.018353 -0.012831 -0.013830 -0.012267  0.024212 -0.033598 -0.014815   

           7         8         9    ...       140       141    

## **FastText**

In [ ]:
pip install gensim

In [ ]:
from gensim.models import FastText
from nltk.tokenize import word_tokenize

# Tokenisasi setiap kalimat menjadi daftar kata-kata
tokenized_sentences = [word_tokenize(sentence) for sentence in df['Kalimat']]

# Buat dan latih model FastText
model_fasttext = FastText(sentences=tokenized_sentences, vector_size=150, window=25, min_count=1, workers=4)


In [ ]:
words=list(model_fasttext.wv.index_to_key)
vector_FastText = [model_fasttext.wv[word] for word in words]
vector_FastText = np.array(vector_FastText)

In [ ]:
words[1:10]

['raja', 'si', 'sang', 'kancil', 'anak', 'puteri', 'istana', 'a', 'orang']

In [ ]:
vec_fasttext=pd.DataFrame(vector_FastText)
print(vec_fasttext)

           0         1         2         3         4         5         6    \
0    -0.178200 -0.297179  0.416828  0.286388 -0.291728  0.215651 -0.890209   
1    -0.148360 -0.247674  0.347771  0.241316 -0.241357  0.179396 -0.746582   
2    -0.146200 -0.237473  0.333681  0.232010 -0.230617  0.174016 -0.713181   
3    -0.429933 -0.717781  1.008868  0.694313 -0.703807  0.517780 -2.159927   
4    -0.153019 -0.253026  0.355569  0.244433 -0.247884  0.182203 -0.760185   
...        ...       ...       ...       ...       ...       ...       ...   
4879 -0.250210 -0.418123  0.586246  0.401740 -0.407185  0.301244 -1.255674   
4880 -0.215448 -0.358584  0.503411  0.346557 -0.351662  0.259069 -1.077641   
4881 -0.254315 -0.421427  0.594860  0.409058 -0.413853  0.304118 -1.272054   
4882 -0.165540 -0.273895  0.381821  0.264429 -0.268952  0.197484 -0.817961   
4883 -0.203141 -0.339625  0.476911  0.328276 -0.332267  0.245810 -1.020019   

           7         8         9    ...       140       141    

In [ ]:
# representasi kata 'malin' pada w2v dan fasttext
word_w2v=model_w2v.wv['malin']
word_fasttext=model_fasttext.wv['malin']

print('Word2Vec:', word_w2v)
print('FastText:', word_fasttext)

Word2Vec: [-0.04396357 -0.03990632 -0.02886433 -0.105688    0.20746963 -0.22636747
 -0.06051996  0.22015017 -0.00626191 -0.01871684  0.24128166  0.02274396
 -0.18446083  0.40127742 -0.3042639   0.05701965  0.02058132  0.05361719
 -0.14848596  0.02885894 -0.21700425  0.02378188  0.1296603  -0.15756293
 -0.09710332  0.05980825 -0.2747287  -0.16497077 -0.05537954 -0.06704765
  0.00820155 -0.08461739 -0.19469045 -0.15234974 -0.08757959 -0.01760192
  0.38036376  0.09263624 -0.00243479 -0.31509435  0.14602499  0.17079453
 -0.19477265  0.06884856  0.22924139 -0.04581464  0.21880174 -0.17425922
  0.17704318 -0.00878582 -0.1227864   0.11435875 -0.12269876  0.19902974
 -0.14028524  0.16899943 -0.23505831  0.13680042 -0.00946294 -0.08574829
 -0.00688985 -0.286034    0.05701552 -0.00899219  0.1680523  -0.01705234
  0.15044071 -0.31996137 -0.31752053 -0.2937455  -0.03174764  0.16265854
 -0.0750914  -0.28581375 -0.15067077  0.29546362 -0.2196337  -0.18551153
 -0.22943309  0.20156302 -0.07453801 -0.0

Pada setiap model, setiap kata akan direpresentasikan dengan dimensi vektor sebanyak 150. Sesuai dengan ukuran vektor yang telah ditetapkan di awal saat model dibangun.

In [ ]:
similar_words_w2v= model_w2v.wv.most_similar('bondowoso', topn=4)
similar_words_fasttext = model_fasttext.wv.most_similar('bondowoso', topn=4)

print(f'Word2Vec - kata serupa dengan "bondowoso":{similar_words_w2v}')
print(f'FastText - kata serupa dengan "bondowoso":{similar_words_fasttext}')

Word2Vec - kata serupa dengan "bondowoso":[('loro', 0.9985447525978088), ('bandung', 0.9984880685806274), ('biji', 0.9984799027442932), ('jin', 0.9984443783760071)]
FastText - kata serupa dengan "bondowoso":[('berbondongbondong', 0.9999478459358215), ('diperintah', 0.9999463558197021), ('jonggrang', 0.9999458193778992), ('memainkan', 0.9999455809593201)]


Pada kasus ini, baik model Word2Vec atau model FastText menunjukan hasil yang sesuai (kata yang mirip memang konteksnya mendekati satu sama lain) walaupun perlu beberapa peningkatan agar hasilnya lebih akurat.

In [ ]:
# untuk model word2vec pre-trained
# inspeksi model
print(f'ukuran vektor: {model_w2v.vector_size}')
print(f'ukuran window: {model_fasttext.window}')
print(f'jumlah kata vektor: {model_w2v.wv}')
print(f'parameter W2V: {model_w2v}')

ukuran vektor: 150
ukuran window: 25
jumlah kata vektor: KeyedVectors<vector_size=150, 4884 keys>
parameter W2V: Word2Vec<vocab=4884, vector_size=150, alpha=0.025>


In [ ]:
# untuk model wfasttext pre-trained
# inspeksi model
print(f'ukuran vektor: {model_fasttext.vector_size}')
print(f'ukuran window: {model_fasttext.window}')
print(f'jumlah kata vektor: {model_fasttext.wv}')
print(f'parameter FastText: {model_fasttext}')

ukuran vektor: 150
ukuran vektor: 25
jumlah kata vektor: FastTextKeyedVectors<vector_size=150, 4884 keys>
parameter FastText: FastText<vocab=4884, vector_size=150, alpha=0.025>


Kedua model, menggunakan ukuran vektor yang sama yaitu 150 dan ukuran _window_ yang sama yaitu 25, ukuran _window_ 25 ditetapkan agar model bisa lebih memahami konteks tanpa menangkap konteks yang terlalu lebar (dengan _window_ yang terlalu besar)